In [2]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import recall_score

In [3]:
#load cleansed data
telecom_df_new = pd.read_excel('DataCleansing_Telecom_Churn.xlsx')
telecom_df_new.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
2,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No
4,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No


In [4]:
#set customerid as index
telecom_df_new.set_index('customerID', inplace=True)
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.20,No
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.50,No
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.50,No


In [5]:
#Convert nominal categorical variables to dummy variables
Gender = pd.get_dummies(telecom_df_new['gender'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, Gender], axis=1)

partner = pd.get_dummies(telecom_df_new['Partner'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, partner], axis=1)

dependents = pd.get_dummies(telecom_df_new['Dependents'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, dependents], axis=1)

lines = pd.get_dummies(telecom_df_new['MultipleLines'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, lines], axis=1)

internet = pd.get_dummies(telecom_df_new['InternetService'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, internet], axis=1)

security = pd.get_dummies(telecom_df_new['OnlineSecurity'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, security], axis=1)

backup = pd.get_dummies(telecom_df_new['OnlineBackup'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, backup], axis=1)

protection = pd.get_dummies(telecom_df_new['DeviceProtection'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, protection], axis=1)

support = pd.get_dummies(telecom_df_new['TechSupport'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, support], axis=1)

tv = pd.get_dummies(telecom_df_new['StreamingTV'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, tv], axis=1)

movies = pd.get_dummies(telecom_df_new['StreamingMovies'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, movies], axis=1)

paperless = pd.get_dummies(telecom_df_new['PaperlessBilling'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, paperless], axis=1)

payment = pd.get_dummies(telecom_df_new['PaymentMethod'], drop_first=True)
telecom_df_new = pd.concat([telecom_df_new, payment], axis=1)

telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,No internet service,Yes,No internet service,Yes,No internet service,Yes,Yes,Credit card (automatic),Electronic check,Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,...,1,0,1,0,1,0,0,1,0,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,...,0,0,0,1,0,0,1,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,...,0,0,0,0,0,0,1,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,...,0,0,0,0,0,1,1,0,1,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,...,0,0,0,0,0,0,0,0,1,0


In [6]:
#Map ordinal values to the Contract feature
telecom_df_new['Contract'] = telecom_df_new['Contract'].map({'Month-to-month': 1, 'One year': 2, 'Two year': 3})
telecom_df_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,No internet service,Yes,No internet service,Yes,No internet service,Yes,Yes,Credit card (automatic),Electronic check,Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,
7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,No internet service,...,1,0,1,0,1,0,0,1,0,0
9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,Yes,...,0,0,0,1,0,0,1,1,0,0
9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,No,...,0,0,0,0,0,0,1,0,0,0
6994-KERXL,Male,0,No,No,4,Yes,No,DSL,No,No,...,0,0,0,0,0,1,1,0,1,0
2181-UAESM,Male,0,No,No,2,Yes,No,DSL,Yes,No,...,0,0,0,0,0,0,0,0,1,0


In [7]:
#create features variable
X = telecom_df_new.drop(columns=['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod', 'Churn'], axis=1).values
X.shape

(5960, 27)

In [8]:
#create target variable
y = telecom_df_new['Churn'].values
y.shape

(5960,)